# ABNB Stock Article extractions

In [1]:
import requests

# api_key = '23f5208b7e2940aeb8b72c67195565b2'
api_key = 'bd9bb272078b41ebb0ee0a905d4a9532'
date_from = '2023-12-11'
date_to = '2024-01-08'
# Expanded list of sources
sources = 'bloomberg,cnbc,reuters,financial-times,techcrunch,the-wall-street-journal,the-verge,business-insider,the-economist,wired,engadget,bbc-news,fortune,techradar'

# Broadening the search query
query = 'IBM OR "International Business Machines" OR "Arvind Krishna" OR "Watson" OR "Cloud computing" OR "Quantum computing" OR "IBM Research" OR "Artificial Intelligence" OR "Blockchain"'

url = f'https://newsapi.org/v2/everything?q={query}&from={date_from}&to={date_to}&sources={sources}&apiKey={api_key}'
response = requests.get(url)
data = response.json()

# Check for 'articles' in response and extract headlines with publication dates
if 'articles' in data:
    news_data = [(article['title'], article['publishedAt'][:10]) for article in data['articles']]
    for headline, date in news_data:
        print(f"Date: {date}, Headline: {headline}")
else:
    print("Error or no articles found. Response:")
    print(data)


Date: 2023-12-21, Headline: The Holy Grail of Quantum Computing Is Finally Here. Or Is it?
Date: 2024-01-04, Headline: Can artificial intelligence help you get fit?
Date: 2024-01-08, Headline: Staying One Step Ahead of Hackers When It Comes to AI
Date: 2023-12-25, Headline: The Hollywood Strikes Stopped AI From Taking Your Job. But for How Long?
Date: 2024-01-01, Headline: The top 10 people in artificial-intelligence healthcare
Date: 2023-12-27, Headline: The top 15 people in enterprise artificial intelligence
Date: 2024-01-01, Headline: Thanks to AI, you don't need a computer science degree to get a job in tech, IBM AI chief says
Date: 2023-12-27, Headline: Omnicom is leveraging AI to optimize workflows with a virtual assistant
Date: 2023-12-22, Headline: The top 5 people in artificial-intelligence sustainability
Date: 2023-12-29, Headline: The top 7 people in artificial-intelligence e-commerce
Date: 2023-12-25, Headline: The top 10 people in artificial-intelligence hardware
Date: 202

# Sentiment Analysis on News Articles

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = AutoModelForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')

# Extract just the headlines from news_data
headlines = [headline for headline, _ in news_data]

# Tokenize and encode headlines
max_length = 512  # You can adjust this based on your requirements
inputs = tokenizer(headlines, padding=True, truncation=True, max_length=max_length, return_tensors="pt")

# Predict sentiment
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

# Pair each sentiment score with its corresponding date
sentiment_data = []
for (headline, date), prediction in zip(news_data, predictions):
    sentiment_data.append({
        'date': date,
        'headline': headline,
        'sentiment_positive': prediction[0].item(),
        'sentiment_neutral': prediction[1].item(),
        'sentiment_negative': prediction[2].item()
    })


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [3]:
sentiment_data

[{'date': '2023-12-21',
  'headline': 'The Holy Grail of Quantum Computing Is Finally Here. Or Is it?',
  'sentiment_positive': 0.9999871253967285,
  'sentiment_neutral': 1.9047136845529167e-07,
  'sentiment_negative': 1.2653823432628997e-05},
 {'date': '2024-01-04',
  'headline': 'Can artificial intelligence help you get fit?',
  'sentiment_positive': 0.8245240449905396,
  'sentiment_neutral': 0.15412722527980804,
  'sentiment_negative': 0.02134871855378151},
 {'date': '2024-01-08',
  'headline': 'Staying One Step Ahead of Hackers When It Comes to AI',
  'sentiment_positive': 0.9951790571212769,
  'sentiment_neutral': 0.0030404203571379185,
  'sentiment_negative': 0.0017806445248425007},
 {'date': '2023-12-25',
  'headline': 'The Hollywood Strikes Stopped AI From Taking Your Job. But for How Long?',
  'sentiment_positive': 0.05843866616487503,
  'sentiment_neutral': 0.00040194709436036646,
  'sentiment_negative': 0.9411593675613403},
 {'date': '2024-01-01',
  'headline': 'The top 10 p

# Historical Stock Data and Sentiment Analysis Combined

In [4]:
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import StandardScaler

# Convert sentiment_data to a DataFrame
sentiment_df = pd.DataFrame(sentiment_data)

# Convert date to datetime and set as index
sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])
sentiment_df.set_index('date', inplace=True)

# Aggregate sentiment scores by date
average_sentiment = sentiment_df.groupby('date').mean()

# Download stock data
ticker_symbol = 'IBM'
stock_data = yf.download(ticker_symbol, start='2023-12-09', end='2024-01-08')

# Selecting the required columns
stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]

# Convert stock_data index to datetime
stock_data.index = pd.to_datetime(stock_data.index)

# Combine stock data with sentiment data
combined_data = stock_data.join(average_sentiment)

# Interpolate missing values
combined_data = combined_data.interpolate(method='time')

# Scale price-related columns
price_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close']
scaler_prices = StandardScaler()
combined_data[price_cols] = scaler_prices.fit_transform(combined_data[price_cols])

# Scale Volume column independently
scaler_volume = StandardScaler()
combined_data['Volume'] = scaler_volume.fit_transform(combined_data[['Volume']])

print(combined_data)

<ipython-input-4-931962714480>:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  average_sentiment = sentiment_df.groupby('date').mean()


[*********************100%%**********************]  1 of 1 completed
                Open      High       Low     Close  Adj Close    Volume  \
Date                                                                      
2023-12-11  0.379440  0.391156  0.483331  0.862865   0.862865  0.896743   
2023-12-12  0.845852  2.327759  1.215884  1.661415   1.661415  0.515444   
2023-12-13  1.715408  1.341464  1.079952  0.936065   0.936065  0.368151   
2023-12-14  0.577068  0.283172 -0.876045  0.463600   0.463600  0.922296   
2023-12-15  0.079054  0.707925  0.551297  0.011087   0.011087  3.295788   
2023-12-18  0.023712  0.160786  0.203908  0.350474   0.350474 -0.269114   
2023-12-19 -0.316202 -0.595139  0.007558 -0.434765  -0.434765 -0.225732   
2023-12-20 -0.719371 -0.940701 -0.981774 -1.439593  -1.439593  0.308252   
2023-12-21 -1.272727 -1.459048 -1.344270 -0.953816  -0.953816 -0.606449   
2023-12-22 -0.869558 -0.501545 -0.234114 -0.048801  -0.048801 -0.870284   
2023-12-26  0.023712  0.146384 

In [5]:
import pandas as pd
df = pd.DataFrame(combined_data)
df

,Open,High,Low,Close,Adj Close,Volume,sentiment_positive,sentiment_neutral,sentiment_negative
Date,,,,,,,,,
2023-12-11,0.379440,0.391156,0.483331,0.862865,0.862865,0.896743,0.000056,0.999936,0.000008
2023-12-12,0.845852,2.327759,1.215884,1.661415,1.661415,0.515444,0.999918,0.000067,0.000015
2023-12-13,1.715408,1.341464,1.079952,0.936065,0.936065,0.368151,0.480045,0.023614,0.496342
2023-12-14,0.577068,0.283172,-0.876045,0.463600,0.463600,0.922296,0.664844,0.142954,0.192202
2023-12-15,0.079054,0.707925,0.551297,0.011087,0.011087,3.295788,0.999373,0.000219,0.000408
2023-12-18,0.023712,0.160786,0.203908,0.350474,0.350474,-0.269114,0.999776,0.000143,0.000081
2023-12-19,-0.316202,-0.595139,0.007558,-0.434765,-0.434765,-0.225732,0.049878,0.500019,0.450103
2023-12-20,-0.719371,-0.940701,-0.981774,-1.439593,-1.439593,0.308252,0.733143,0.236099,0.030758
2023-12-21,-1.272727,-1.459048,-1.344270,-0.953816,-0.953816,-0.606449,0.799787,0.000041,0.200172


# Transformer Architecture

In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from tensorflow.keras.callbacks import EarlyStopping

selected_features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sentiment_positive', 'sentiment_neutral', 'sentiment_negative']
data = combined_data[selected_features]

train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.15)
train_data = data.iloc[:train_size]
val_data = data.iloc[train_size:train_size + val_size]
test_data = data.iloc[train_size + val_size:]

# Prepare features and target for model training
X_train = train_data.drop('Close', axis=1)
y_train = train_data['Close']
X_val = val_data.drop('Close', axis=1)
y_val = val_data['Close']
X_test = test_data.drop('Close', axis=1)
y_test = test_data['Close']

# Define and compile the neural network model
model = Sequential()
model.add(Dense(32, activation='LeakyReLU', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(16, activation='LeakyReLU'))
model.add(Dense(1))
model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    X_train, y_train,
    epochs=50,  # Set back to 50 or an arbitrarily large number
    batch_size=8,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]  # Add the early stopping callback here
)

train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_loss) + 1)

# Convert the epochs range object to a list
epochs_list = list(epochs)

# Plotting the training and validation loss
loss_fig = go.Figure()
loss_fig.add_trace(go.Scatter(x=epochs_list, y=train_loss, mode='lines', name='Training Loss'))
loss_fig.add_trace(go.Scatter(x=epochs_list, y=val_loss, mode='lines', name='Validation Loss'))
loss_fig.update_layout(title='Training and Validation Loss per Epoch',
                       xaxis_title='Epochs',
                       yaxis_title='Loss',
                       legend_title='Type')
loss_fig.show()

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print(f'MSE: {mse}, MAE: {mae}, R^2: {r2}')

# Reshape y_test and predictions for inverse transform
y_test_reshaped = y_test.values.reshape(-1, 1)
predictions_reshaped = predictions.reshape(-1, 1)

num_scaled_cols = 5

# Create separate dummy arrays for inverse scaling
dummy_array_y_test = np.zeros((len(y_test_reshaped), num_scaled_cols))
dummy_array_predictions = np.zeros((len(predictions_reshaped), num_scaled_cols))

# Fill in the 'Close' column values in the dummy arrays
# Assuming 'Close' is the last of the scaled columns
dummy_array_y_test[:, -1] = y_test_reshaped.flatten()
dummy_array_predictions[:, -1] = predictions_reshaped.flatten()

# Inverse transform the 'Close' prices using the dummy arrays
y_test_original = scaler_prices.inverse_transform(dummy_array_y_test)[:, -1]
predictions_original = scaler_prices.inverse_transform(dummy_array_predictions)[:, -1]

# Extracting testing dates
test_dates = test_data.index

# Plotting with Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_dates, y=y_test_original, mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=test_dates, y=predictions_original, mode='lines', name='Predicted'))
fig.update_layout(title='Actual vs Predicted Stock Prices', xaxis_title='Date', yaxis_title='Stock Price', legend_title='Legend')
fig.show()

Epoch 1/50
2/2 [==============================] - 1s 198ms/step - loss: 0.4125 - val_loss: 0.1441
Epoch 2/50
2/2 [==============================] - 0s 35ms/step - loss: 0.3451 - val_loss: 0.1941
Epoch 3/50
2/2 [==============================] - 0s 38ms/step - loss: 0.1620 - val_loss: 0.1062
Epoch 4/50
2/2 [==============================] - 0s 37ms/step - loss: 0.1600 - val_loss: 0.0049
Epoch 5/50
2/2 [==============================] - 0s 35ms/step - loss: 0.1141 - val_loss: 0.0350
Epoch 6/50
2/2 [==============================] - 0s 53ms/step - loss: 0.1231 - val_loss: 0.0128
Epoch 7/50
2/2 [==============================] - 0s 34ms/step - loss: 0.0487 - val_loss: 0.0073
Epoch 8/50
2/2 [==============================] - 0s 35ms/step - loss: 0.0940 - val_loss: 0.0079
Epoch 9/50
2/2 [==============================] - 0s 45ms/step - loss: 0.0649 - val_loss: 0.0077
Epoch 9: early stopping


1/1 [==============================] - 0s 69ms/step
MSE: 0.02299279924097522, MAE: 0.11174042066056815, R^2: 0.9318840873826046


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 833 (3.25 KB)
Trainable params: 833 (3.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
